In [ ]:
import datetime
from typing import Any

import dotenv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pendulum
from octopus_stats import (
    ConsumptionRecord,
    OctoAPIConfig,
    OctoAPIReader,
)
from zappi_stats.zappi_api_reader import (
    MyenergiApiConfig,
    ZappiApiReader,
    ZappiUsageByMinuteRecordRaw,
)


In [ ]:
def get_consumption_data(cr: ConsumptionRecord) -> dict[str, Any]:
    cd: dict[str, Any] = {}
    cd["start_utc"] = cr.interval_start.astimezone(datetime.UTC)
    cd["end_utc"] = cr.interval_end.astimezone(datetime.UTC)
    cd["octo_consumed_kwh"] = cr.consumption
    cd["start_local"] = cr.interval_start.replace(tzinfo=None)
    cd["start_utc_offset"] = cr.interval_start.utcoffset()
    cd["end_local"] = cr.interval_end.replace(tzinfo=None)
    cd["end_utc_offset"] = cr.interval_end.utcoffset()

    return cd


def joules_to_kwh(joules: int) -> float:
    return round(joules / 3_600_000, 5)


def get_charging_data(cr: ZappiUsageByMinuteRecordRaw) -> dict[str, Any]:
    cd: dict[str, Any] = {}
    cd["start_utc"] = cr.interval_start
    cd["imported_kwh"] = joules_to_kwh(cr.imp)
    cd["exported_kwh"] = joules_to_kwh(cr.exp)
    cd["ev_charged_kwh"] = joules_to_kwh(sum([cr.h1b, cr.h1d, cr.h2b, cr.h2d, cr.h3b, cr.h3d]))
    cd["volts"] = cr.v1 / 10
    cd["frequency"] = cr.frq / 100

    return cd

In [ ]:
my_tz = "Europe/London"
PERIOD_START = pendulum.datetime(2023, 7, 13, 0, 0, tz=my_tz)
PERIOD_END=pendulum.datetime(2023, 12, 30, 0, 0, tz=my_tz)

In [ ]:
dotenv.load_dotenv()
config = OctoAPIConfig.from_env()
octo_api_reader = OctoAPIReader(config)

consumption = octo_api_reader.get_consumption(
    account_number=config.account_number,
    start=PERIOD_START,
    end=PERIOD_END,
)

df_consumption = pd.DataFrame([get_consumption_data(cr) for cr in consumption])

df_consumption["duration"] = df_consumption["end_utc"] - df_consumption["start_utc"]
df_consumption["date_local"] = df_consumption["start_local"].dt.normalize()
df_consumption["dayofyear_local"] = df_consumption["start_local"].dt.dayofyear
df_consumption["dayofweek_local"] = df_consumption["start_local"].dt.dayofweek
df_consumption["hourofday_local"] = df_consumption["start_local"].dt.hour

df_consumption

In [ ]:
df_consumption.dtypes

In [ ]:
dotenv.load_dotenv()
config = MyenergiApiConfig.from_env()
api = ZappiApiReader(config)
api.connect()
x = [
    get_charging_data(x)
    for x in api.get_data(
        start=PERIOD_START,
        end=PERIOD_END,
    )
]
df = pd.DataFrame(x)
df

In [ ]:
df_charger_consumption = (
    df.groupby(
        pd.cut(
            x=df["start_utc"],
            bins=df_consumption["start_utc"],
            labels=df_consumption.iloc[:-1]["start_utc"],
            include_lowest=True,
            right=False,
        ),
        observed=False,
    ).agg({"ev_charged_kwh": "sum", "imported_kwh": "sum", "exported_kwh": "sum"}).reset_index()
)
df_charger_consumption["start_utc"] = pd.to_datetime(df_charger_consumption["start_utc"])
# Select and display just the rows with a non-zero value
df_charger_consumption.loc[df_charger_consumption["ev_charged_kwh"] > 0]

In [ ]:
df_half_hour_consumption = df_consumption.merge(df_charger_consumption, on="start_utc")
df_half_hour_consumption["domestic_consumed_kwh"] = (
    df_half_hour_consumption["imported_kwh"]
    - df_half_hour_consumption["ev_charged_kwh"]
)
df_half_hour_consumption

In [ ]:
df_consumption_by_day = df_half_hour_consumption.groupby(
    df_half_hour_consumption["date_local"], as_index=False
).agg(
    {
        "octo_consumed_kwh": "sum",
        "ev_charged_kwh": "sum",
        "imported_kwh": "sum",
        "domestic_consumed_kwh": "sum",
    }
)

In [ ]:
df_consumption_by_day["ratio"] = df_consumption_by_day["imported_kwh"] / df_consumption_by_day["octo_consumed_kwh"]
df_consumption_by_day

In [ ]:
# Plot histogram: domestic + ev (stacked) next to consumed - the bars should be the same height
fig, ax = plt.subplots(figsize=(12, 4))
width = 0.3
days = np.arange(len(df_consumption_by_day["date_local"]))
ax.bar(days, df_consumption_by_day["domestic_consumed_kwh"], width=width)
ax.bar(
    days,
    df_consumption_by_day["ev_charged_kwh"],
    bottom=df_consumption_by_day["domestic_consumed_kwh"],
    width=width,
)
ax.bar(days + width, df_consumption_by_day["imported_kwh"], width=width)
_ = ax.set_xticks(
    days,
    df_consumption_by_day["date_local"],
    rotation=40,
    ha="right",
    rotation_mode="anchor",
)

In [ ]:
df_consumption_by_time_of_day = df_half_hour_consumption.groupby(
    df_half_hour_consumption["hourofday_local"], as_index=False
).agg(
    {
        "octo_consumed_kwh": "mean",
        "ev_charged_kwh": "mean",
        "imported_kwh": "mean",
        "domestic_consumed_kwh": "mean",
    }
)

df_consumption_by_time_of_day

In [ ]:
x = df_consumption_by_time_of_day["hourofday_local"]
plt.plot(x, df_consumption_by_time_of_day["ev_charged_kwh"], label="EV")
plt.plot(x, df_consumption_by_time_of_day["domestic_consumed_kwh"], label="domestic")
plt.xlabel("Hour")
plt.ylabel("Consumption")
plt.legend()
plt.title("EV and Domestic Usage")
plt.show()